In [ ]:
!pip install pydot
!pip install pydotplus
!apt install graphviz

# Keras
Mediante la librería **Tensorflow** se pueden definir, entrenar y validar modelos de *Machine Learning*. Para ello hay que definir las entradas del modelo, sus variables (*parámetros*), el algoritmo de optimización y la función de coste. Si bien estas tareas no entrañan una dificultad elevada, el proceso es tedioso.

**Keras**, un API de alto nivel para construir y entrenar modelos en Tensorflow nos ofrece las mismas posibilidades aunque a un nivel de abstracción mayor, facilitando la tarea.

Para comprobar su funcionamiento, vamos a construir y entrenar algunos modelos para resolver problemas clásicos de aprendizaje supervisado.

## Caso de estudio: precio de las casas de Boston

Este conjunto de datos es bien conocido entre la comunidad. Se encuentra disponible en la librería *Scikit-Learn*. El conjunto de datos cuenta con 13 *features* numéricas sin estandarizar. La variable de salida es el precio en miles de dólares.

In [ ]:
from sklearn.datasets import load_boston
from sklearn.utils import shuffle
boston = load_boston()
X, y = shuffle(boston.data, boston.target, random_state=1337)

**Keras** nos permite crear un modelo de aprendizaje de tipo red neuronal artificial. La red más sencilla es aquella compuesta por una capa de entrada, una de salida y varias capas ocultas. Para construir una red de estas características con **Keras** podemos usar el tipo `Sequential` e ir añadiendo las distintas capas.

In [ ]:
import tensorflow as tf
tf.keras.backend.clear_session()

def generate_model_boston():
    return tf.keras.Sequential([
        tf.keras.Input(shape=(X.shape[1],)),
        tf.keras.layers.Dense(13, activation='relu'),
        tf.keras.layers.Dense(6, activation='relu'),
        tf.keras.layers.Dense(3, activation='relu'),
        tf.keras.layers.Dense(1),
    ])

model_boston = generate_model_boston()
print(model_boston.summary())

Ademas de la función `.summary()` que nos devuelve una representación textual de la red para poder imprimirla por pantalla, tenemos la posibilidad de representar la red gráficamente:

In [ ]:
from keras.utils import plot_model

plot_model(model_boston, show_shapes=True)

Una vez definido el modelo hay que compilarlo para poder usarlo. Además, en el momento de la compilación del modelo tenemos que definir la **función de coste**, el **optimizador** y las **métricas** que queremos calcular. A partir de este momento ya podemos entrenar el modelo mediante `fit()` alimentándolo con el conjunto de datos de entrenamiento. Observa que el propio método de ajuste `fit()` tiene un parámetro para realizar el split en entrenamiento y test. Date cuenta también que no estamos utilizando *batches*, sino el conjunto de datos completo en cada *epoch* del entrenamiento.

In [ ]:
with tf.device('/device:GPU:1'):
    tf.keras.backend.clear_session()
    model_boston = generate_model_boston()
    model_boston.compile(
        optimizer='adam',
        loss=tf.keras.losses.mean_squared_error,
        metrics=[
            tf.keras.metrics.MeanAbsoluteError(),
            tf.keras.metrics.RootMeanSquaredError()
        ]
    )
    history = model_boston.fit(X, y, epochs=25, verbose=0, validation_split=0.2)

La librería **Keras** también nos permite validar el modelo de forma muy sencilla, ejecutando `evaluate()`. Si bien en este ejemplo estamos usando el dataset completo, esta función sería útil si nos hubiésemos guardado un conjunto de test.

In [ ]:
model_boston.evaluate(X, y)

También podemos usar la información histórica que devuelve el proceso de ajuste (método `fit()`) para representar gráficamente cómo evoluciona el valor de la función de pérdida así como otras métricas tanto para el conjunto de datos de entrenamiento como para el de validación.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='MSE (train)')
plt.plot(history.history['val_loss'], label='MSE (validation)')
plt.title('MSE for Boston houses (log-scale)')
plt.xlabel('epoch')
plt.legend(loc="upper right")
plt.yscale('log')
plt.show()

Podemos hacer lo mismo con el resto de métricas de evaluación que configuramos en el momento de compilación del modelo. En este caso representamos la evolución del MAE tanto para entrenamiento como para validación.

In [ ]:
plt.plot(history.history['mean_absolute_error'], label='MAE (train)')
plt.plot(history.history['val_mean_absolute_error'], label='MAE (val)')
plt.title('MAE for Boston houses (Log-scale)')
plt.xlabel('epoch')
plt.legend(loc="upper right")
plt.yscale('log')
plt.show()

### Entrenamiento por lotes para reducir el *overfitting*
Podemos observar que las curvas de entrenamiento y validación se cruzan alrededor del *epoch* 12, indicando un sobreajuste del modelo. Esto es normal con las **redes neuronales artificiales** y, además, no hemos utilizado entrenamiento por lotes. Esto provoca que en cada *epoch* se utilice el dataset completo para ajustar los pesos, dando lugar al **overfitting**. Vamos a estudiar cómo afecta el tamaño del lote al sobreajuste del modelo

En primer lugar vamos a crear una función para dibujar la gráfica de la evolución de las métricas de calidad, así es más cómodo:

In [ ]:
def plot_train_val(history, metric='loss', y_scale='linear'):
    plt.plot(history.history[metric], label=metric + ' (train)')
    plt.plot(history.history['val_' + metric], label=metric + ' (val)')
    plt.title(f'{metric} ({y_scale}-scale)')
    plt.xlabel('epoch')
    plt.legend(loc="upper right")
    plt.yscale(y_scale)
    plt.show()

Ahora vamos a reajustar el modelo utilizando entrenamiento por lotes. Para ello proporcionaremos el tamaño del lote al parámetro `batch_size` del método `fit()`:

In [ ]:
with tf.device('/device:GPU:1'):
    tf.keras.backend.clear_session()
    model_boston = generate_model_boston()
    model_boston.compile(
        optimizer='adam',
        loss=tf.keras.losses.mean_squared_error,
        metrics=[
            tf.keras.metrics.MeanAbsoluteError(),
            tf.keras.metrics.RootMeanSquaredError()
        ]
    )
    history = model_boston.fit(X, y, epochs=25, batch_size=25, verbose=0, validation_split=0.2)
plot_train_val(history, metric='mean_absolute_error')

## Caso de estudio: MNIST

El siguiente dataset que vamos a trabajar es el clásico MNIST de dígitos manuscritos. Es un problema de clasificación que consiste en identificar el dígito de 0 a 9 analizando la imagen escaneada.

### API funcional vs. secuencial de Keras
En el ejemplo anterior hemos utilizado la *API secuencial* de Keras para definir el modelo. Esta API obtiene su nombre de la forma de construir los modelos añadiendo capas, una detrás de otra. Aunque muy sencilla de utilizar, esta API tiene un gran inconveniente, y es que los modelos definidas con ella **siempre son modelos lineales**, es decir, una secuencia interconectada de capas desde la entrada a la salida.

*Keras* nos ofrece una alternativa mucho más flexible que permite diseñar redes como un grafo acíclico dirigido: la **API funcional**. Vamos a ver cómo se usa esta API definiendo un modelo para resolver el problema de clasificación que se nos plantea.

In [ ]:
tf.keras.backend.clear_session()

def generate_model_mnist():
    inputs = tf.keras.Input(shape=(784,)) # Las imágenes de MNIST son de 28x28 píxeles

    x = tf.keras.layers.Dense(64, activation='relu')(inputs) # Primera capa densa conectada a 'inputs'
    x = tf.keras.layers.Dense(64, activation='relu')(x) # Segunda capa densa conectada a la primera 'x'

    outputs = tf.keras.layers.Dense(10, activation='softmax')(x) # Capa de salida con 10 neuronas, una por clase
    return tf.keras.Model(inputs=inputs, outputs=outputs, name="mnist")

plot_model(generate_model_mnist(), show_layer_names=True, show_shapes=True)

Como podemos observar, la forma de construirse que tiene el modelo nos permite topologías no lineales, varias entradas, etc. Procedemos ahora a cargar el *dataset*. Hacemos una normalización de los valores de las variables para pasar de un nivel de escala de grises de 0 a 255 a un valor continuo entre 0 y 1.

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(60000, 784).astype("float32") / 255
x_test = x_test.reshape(10000, 784).astype("float32") / 255

Ya solo nos queda compilar y entrenar el modelo. Es interesante recalcar que, dado que el problema es de clasificación, tenemos que usar una función de pérdida adecuada para este tipo de problemas. En nuestro caso usaremos **entropía cruzada categórica**.

In [ ]:
with tf.device('/device:GPU:0'):
    tf.keras.backend.clear_session()
    model_mnist = generate_model_mnist()
    model_mnist.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['accuracy']
    )
    history = model_mnist.fit(x_train, y_train, batch_size=64, epochs=10, validation_split=0.2)

In [ ]:
plot_train_val(history, metric='accuracy')

Evaluamos el modelo con el conjunto de datos de test:

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

with tf.device('/device:GPU:0'):
    y_pred = np.argmax(model_mnist.predict(x_test, batch_size=64, verbose=0), axis=1)

print(classification_report(y_test, y_pred))

## Caso de estudio: calidad de los vinos
En esta ocasión tenemos que predecir la nota (0 a 10) que un experto catador otorgará a un vino en función de sus propiedades fisicoquímicas. El conjunto de datos está incluido en **Tensorflow**.

In [ ]:
import tensorflow_datasets as tfds
ds, ds_info = tfds.load(
    'wine_quality',
    split='train',
    shuffle_files=True,
    with_info=True,
)
print(ds_info)

Convertimos a Pandas Dataframe para que sea más cómodo de procesar.

In [ ]:
from sklearn.utils import shuffle
raw_data = tfds.as_dataframe(ds, ds_info)
raw_data = shuffle(raw_data)
Y = raw_data['quality']
X = raw_data.drop('quality', axis=1)

En primer lugar tendríamos que normalizar/estandarizar las variables de entrada. Vamos a utilizar una capa de **Keras** que hace precisamente eso, por lo cual integraremos la normalización directamente en el modelo, justo a continuación de la salida.

In [ ]:
tf.keras.backend.clear_session()

def generate_model_wines():
    inputs = tf.keras.layers.Input((11,))
    #normalize = tf.keras.layers.experimental.preprocessing.Normalization()(inputs)
    x = tf.keras.layers.Dense(16, activation='tanh')(inputs)
    x = tf.keras.layers.Dense(16, activation='tanh')(x)
    outputs = tf.keras.layers.Dense(11, activation='softmax')(x) # Notas de 0 a 10, ambas incluidas
    return tf.keras.Model(inputs=inputs, outputs=outputs, name="wine_quality")

plot_model(generate_model_wines(), show_layer_names=True, show_shapes=True)

In [ ]:
with tf.device('/device:GPU:0'):
    tf.keras.backend.clear_session()
    model_wines = generate_model_wines()
    model_wines.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.Adam(0.001),
        metrics=['accuracy']
    )
    history = model_wines.fit(X.values, Y.values, batch_size=16, epochs=25, verbose=0, validation_split=0.2)
plot_train_val(history)

### Pipelines y variables de entrada

Podemos observar que planteando el problema como clasificación no obtenemos buenos resultados. Hay que pensar que la nota que se asigna a un vino tiene un orden, ya que no es lo mismo equivocarse prediciendo una nota de 2 cuando la real era un 10, que predecir un 9. Es por ello que vamos a plantear el problema como si de una regresión se tratase, devolviendo una salida continua que representa la votación predicha por el modelo.

Además, vamos a aprovechar este cambio para plantear el uso de *pipelines* para conectar los *datasets* al modelo, además de utilizar una entrada individual por cada una de las *features* del *dataset*.

En primer lugar, cargamos el conjunto de datos `wine_quality` utilizando tfds.load(), y convertimos la variable objetivo en `float`, ya que ahora el problema será de regresión y no de clasificación. A continuación, barajamos el conjunto de datos y lo dividimos en conjuntos de entrenamiento y de test. Tomamos los primeros `test_size` observaciones como la división de entrenamiento, y el resto como la división de test. Todo ello lo hacemos utilizando un **pipeline** de *Tensorflow*:

In [ ]:
to_regression = lambda x, y: (x, tf.cast(y, tf.float32))

def get_train_and_test_splits(train_size, batch_size=1):
    dataset = (
        tfds.load(name="wine_quality", as_supervised=True, split="train")
        .map(to_regression)
        .prefetch(buffer_size=dataset_size)  # El dataset es pequeño y cabe entero en memoria
        .cache()
    )

    train_dataset = (
        dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    )
    test_dataset = dataset.skip(train_size).batch(batch_size)

    return train_dataset, test_dataset

Como hemos dicho antes, vamos a construir un modelo en el cual cada *feature* es una entrada distinta del mismo. Para ello tenemos que darle nombres a las entradas. El resto del modelo será similar al que hemos definido antes, solo que ahora la capa de salida constará de una única neurona con una función de activación continua.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

FEATURE_NAMES = [
    "fixed_acidity",
    "volatile_acidity",
    "citric_acid",
    "residual_sugar",
    "chlorides",
    "free_sulfur_dioxide",
    "total_sulfur_dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs

def regression_wine():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)
    for units in [16, 16]:
        features = layers.Dense(units, activation="sigmoid")(features)
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model
plot_model(regression_wine(), show_layer_names=True, show_shapes=True)

Vamos a hacer una partición de entrenamiento y test en 80% y 20%, respectivamente.

In [ ]:
dataset_size = 4898
batch_size = 256
train_size = int(dataset_size * 0.80)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)

Y procedemos al entrenamiento y validación del modelo construido.

In [ ]:
with tf.device('/device:GPU:0'):
    tf.keras.backend.clear_session()
    reg_wines = regression_wine()
    reg_wines.compile(
        loss=tf.keras.losses.MeanSquaredError(),
        optimizer=tf.keras.optimizers.Adam(0.001),
        metrics=[keras.metrics.RootMeanSquaredError()],
    )
    history = reg_wines.fit(train_dataset, epochs=50, validation_data=test_dataset, verbose=0)
plot_train_val(history)

In [ ]:
plot_train_val(history, metric='root_mean_squared_error')

---

Creado por **Raúl Lara Cabrera** (raul.lara@upm.es)

<img src="https://licensebuttons.net/l/by-nc-sa/3.0/88x31.png">